# Day 19 (2023-12-19), Exercise a

In [1]:
with open("input_19_dummy.txt", "r") as fh:
    lines = fh.readlines()
# Remove \n characters
lines = [l.strip() for l in lines]

In [2]:
# Prepare input
rules = {}
numbers = []
linebreak_idx = [i for i,l in enumerate(lines) if l == ''][0]
for l in lines[:linebreak_idx]:
    key, value = l.split('{')
    value = value.removesuffix('}').split(',')
    value = [tuple(v.split(':')) if ':' in v else v for v in value]
    rules[key] = value

for l in lines[linebreak_idx+1:]:
    l = l.removeprefix('{').removesuffix('}').split(',')
    l = [tuple(li.split('=')) for li in l]
    numbers.append({x[0]:x[1] for x in l})

In [3]:
s_accepted = 0
for n in numbers:
    key = 'in'
    while key not in ['A', 'R']:
        rule = rules[key]
        for r in rule:
            if type(r) == tuple:
                t = r[0].replace('x', n['x']).replace('m', n['m']).replace('a', n['a']).replace('s', n['s'])
                if eval(t):
                    key = r[1]
                    break
            else:
                key = r
    if key == 'A':
        s_accepted += int(n['x']) + int(n['m']) + int(n['a']) + int(n['s'])
s_accepted

19114

# Day 19 (2023-12-19), Exercise b

In [26]:
# Determine all rules that end in 'A'
start_conditions = []
for rule in rules.items():
    for i, r in enumerate(rule[1]):
        if type(r) == tuple:
            if r[1] == 'A':
                start_conditions.append((rule[0], r[0]))
        else:
            # Assumption: always happens as last element
            if r == 'A':
                cs = ''
                for rj in rule[1][:i]:
                    cs += f'not {rj[0]} and '
                start_conditions.append((rule[0], cs.strip().removesuffix('and').strip()))
start_conditions

[('px', 'm>2090'),
 ('pv', 'not a>1716'),
 ('lnx', 'm>1548'),
 ('lnx', 'not m>1548'),
 ('rfg', 'not s<537 and not x>2440'),
 ('qs', 's>3448'),
 ('qkq', 'x<1416'),
 ('crn', 'x>2662'),
 ('hdj', 'm>838')]

In [31]:
# Compose list of conditions that all lead to 'A' (note: each sub-list should be AND)
conditions = []
for sc in start_conditions:
    key = sc[0]
    and_conditions = [sc[1]]
    while key != 'in':
        for rule in rules.items():
            for i, r in enumerate(rule[1]):
                if type(r) == tuple:
                    if r[1] == key:
                        and_conditions.append(r[0])
                        key = rule[0]
                else:
                    # Assumption: always happens as last element
                    if r == key:
                        cs = ''
                        for rj in rule[1][:i]:
                            cs += f'not {rj[0]} and '
                        and_conditions.append(cs.strip().removesuffix('and').strip())
                        key = rule[0]
    conditions.append(and_conditions)

[['m>2090', 's<1351'],
 ['not a>1716', 'not m>838', 'm<1801', 'not s<1351'],
 ['m>1548', 'not s>3448', 's>2770', 'not s<1351'],
 ['not m>1548', 'not s>3448', 's>2770', 'not s<1351'],
 ['not s<537 and not x>2440', 'not a<2006 and not m>2090', 's<1351'],
 ['s>3448', 's>2770', 'not s<1351'],
 ['x<1416', 'a<2006', 's<1351'],
 ['x>2662', 'not x<1416', 'a<2006', 's<1351'],
 ['m>838', 'm<1801', 'not s<1351']]

In [64]:
from math import prod
total_possibilities = 0
all_xmas = []
for condition in conditions:
    xmas = {
    'x': (1, 4000),
    'm': (1, 4000),
    'a': (1, 4000),
    's': (1, 4000),
    }
    for and_cond in condition:
        and_cond2 = and_cond.split(' and ')
        for a in and_cond2:
            if len(a) != len(a.removeprefix('not')):
                # FALSE-ie
                b = a.removeprefix('not ')
                if '<' in b:
                    r = xmas[b[0]]
                    xmas[b[0]] = (max(r[0], int(b.split('<')[1])), r[1])
                if '>' in b:
                    r = xmas[b[0]]
                    xmas[b[0]] = (r[0], min(r[1], int(b.split('>')[1])))
            else:
                # TRUE-ie
                if '<' in a:
                    r = xmas[a[0]]
                    xmas[a[0]] = (r[0], min(r[1], int(a.split('<')[1])-1))
                if '>' in a:
                    r = xmas[a[0]]
                    xmas[a[0]] = (max(r[0], int(a.split('>')[1])-1), r[1])
    total_possibilities += prod([i[1]-i[0]+1 for i in xmas.values()])
    all_xmas.append(xmas)
total_possibilities # TODO currently not yet distinct; i.e., ranges overlap

# Remove overlapping ranges - TODO
all_xmas_set = []
total_possibilities = 0
for xmax in all_xmas:
    if xmas not in all_xmas_set:
        all_xmas_set.append(xmas)
        total_possibilities += prod([i[1]-i[0]+1 for i in xmas.values()])
total_possibilities
all_xmas

[{'x': (1, 4000), 'm': (2089, 4000), 'a': (1, 4000), 's': (1, 1350)},
 {'x': (1, 4000), 'm': (1, 838), 'a': (1, 1716), 's': (1351, 4000)},
 {'x': (1, 4000), 'm': (1547, 4000), 'a': (1, 4000), 's': (2769, 3448)},
 {'x': (1, 4000), 'm': (1, 1548), 'a': (1, 4000), 's': (2769, 3448)},
 {'x': (1, 2440), 'm': (1, 2090), 'a': (2006, 4000), 's': (537, 1350)},
 {'x': (1, 4000), 'm': (1, 4000), 'a': (1, 4000), 's': (3447, 4000)},
 {'x': (1, 1415), 'm': (1, 4000), 'a': (1, 2005), 's': (1, 1350)},
 {'x': (2661, 4000), 'm': (1, 4000), 'a': (1, 2005), 's': (1, 1350)},
 {'x': (1, 4000), 'm': (837, 1800), 'a': (1, 4000), 's': (1351, 4000)}]